In [106]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""
import sys

In [107]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *
from twinews.models.ranking import *
from twinews.evaluation.utils import *

In [108]:
from nlptools.basics import *
from nltk.stem import WordNetLemmatizer 
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import LdaMulticore
from gensim.test.utils import common_corpus, common_dictionary
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
import gensim
from math import log2
from math import sqrt
from numpy import asarray

In [109]:
from twinews.yfnotebooks.convnet.graph import Graph
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from twinews.yfnotebooks.load_data import load_char_data,char_index

In [110]:
isNotebook = '__file__' not in locals()
TEST = isNotebook

In [111]:
logger = Logger(tmpDir('logs') + "/Convnet.log") if isNotebook else Logger("Convnet-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

In [112]:
modelName = "Convnet"

In [113]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["TF_CPP_MIN_LOG_LEVEL’"] = "3"
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# config

In [114]:
config = \
{
    'splitVersion': 2,
    'testVersion':2,
    
    # for input data
    'maxUsers': 500 if TEST else None, # Sub-sampling
    'maxDocuments': None,
    'useExtraNews': 0 if TEST else None, # None = unlimited, 0 = no extra news
    'minDF': 1 / 2000, # Remove words that have a document frequency ratio lower than 1 / 500
    'maxDF': 20, # Remove top 300 voc elements
    'lowercase': True,
    'doLemmatization': True,
    #'useTFIDF': True,
    
   # for model    
    'seq_length' : 100,
#     'char_embedding_size': 100,
    'learning_rate': 0.0005,
    'keep_prob': 0.7,
    'vocab_size': 3029,
    'class_size': 2,
    'epochs': 100,
    'batch_size': 1000,
    
    # for ouput
    'similarity': 'cosine',

#     'historyRef': 0.3, # 1, 1.0, 0.5, 0.3, 3, 10
}

# get data

In [115]:
# Getting users and news
evalData = getEvalData(config['splitVersion'], maxExtraNews=0,
                       maxUsers=config['maxUsers'])
(trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
(evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
 evalData['testNews'], evalData['candidates'], evalData['extraNews'])
bp(evalData.keys(), 5, logger)
log(b(evalData['meta'], 5), logger)

--> tic: 29.55s | message: Eval data loaded


{ candidates, extraNews, meta, testNews, testUsers, trainNews, trainUsers }
{ 'created': 2020.03.24-14.28.06, 'endDate': 2018-01-15, 'id': 2, 'ranksLength': 1000, 'splitDate': 2017-12-25, 'startDate': 2017-10-01, 'testMaxNewsPerUser': 97, 'testMeanNewsPerUser': 7.22, 'testMinNewsPerUser': 2, 'testNewsCount': 71781, 'totalNewsAvailable': 570210, 'trainMaxNewsPerUser': 379, 'trainMeanNewsPerUser': 26.48, 'trainMinNewsPerUser': 8, 'trainNewsCount': 237150, 'usersCount': 15905 }


--> toc total duration: 29.93s | message: Got Twinews evaluation data


In [116]:
trainNewsList = shuffle(list(trainNews), seed=0)
testNewsList = shuffle(list(testNews), seed=0)

In [117]:
# get all the url->title dict
newsList = trainNewsList + testNewsList
log(str(len(newsList)) + " urls for newsList:\n" + b(newsList), logger=logger)

83649 urls for newsList:
[
  https://www.bloomberg.com/news/articles/2017-11-09/u-s-energy-regulator-lays-out-plan-to-keep-at-ris,
  http://fb.me/1n603rFoK,
  ...,
  http://www.washingtonexaminer.com/article/2645213,
  https://www.nytimes.com/2018/01/03/technology/personaltech/technology-future-journalism.html
]


In [118]:
titles = getNewsTitles(newsList)

  100% [====================] (total duration: 50m 33.96s, mean duration: 0.036s)


In [119]:
titlesToken = []
for i in pb(list(range(len(titles))), logger=logger, message="tokenizing"):
    titlesToken.append(wordTokenize(titles[i]))
bp(titlesToken,2)

tokenizing   0% [                    ]
tokenizing   9% [=                   ] (9.361s left)
tokenizing  19% [===                 ] (8.241s left)
tokenizing  29% [=====               ] (7.211s left)
tokenizing  39% [=======             ] (6.196s left)
tokenizing  49% [=========           ] (5.171s left)
tokenizing  59% [===========         ] (4.241s left)
tokenizing  69% [=============       ] (3.168s left)
tokenizing  79% [===============     ] (2.106s left)
tokenizing  89% [=================   ] (1.051s left)
tokenizing  99% [=================== ] (0.001s left)
tokenizing 100% [====================] (total duration: 10.5s, mean duration: 0s)


[ [ U.S., Energy, ..., -, Bloomberg ], [ Take, religion, ..., -, Vox ], ..., [ White, House, ..., book, ' ], [ Tapping, Technology, ..., York, Times ] ]


In [120]:
for i in pb(list(range(len(titlesToken))), logger=logger, message="Lower casing"):
    if titlesToken[i] == None:
        print(titlesToken[i])
        continue
    else:
        for u in range(len(titlesToken[i])):
            titlesToken[i][u] = titlesToken[i][u].lower()
bp(titlesToken, logger)

Lower casing   0% [                    ]
Lower casing   9% [=                   ] (1.17s left)


None


Lower casing  19% [===                 ] (1.08s left)
Lower casing  29% [=====               ] (0.933s left)


None
None
None
None


Lower casing  39% [=======             ] (0.795s left)
Lower casing  49% [=========           ] (0.66s left)
Lower casing  59% [===========         ] (0.526s left)


None
None
None


Lower casing  69% [=============       ] (0.394s left)
Lower casing  79% [===============     ] (0.262s left)


None
None
None


Lower casing  89% [=================   ] (0.131s left)
Lower casing  99% [=================== ] (0s left)
Lower casing 100% [====================] (total duration: 1.31s, mean duration: 0s)
[ [ u.s., energy, ..., -, bloomberg ], [ take, religion, ..., -, vox ], ..., [ white, house, ..., book, ' ], [ tapping, technology, ..., york, times ] ]


In [121]:
lemmatizer = WordNetLemmatizer()
pbar = ProgressBar(len(titlesToken), logger=logger, message="Lemmatization")
for i in range(len(titlesToken)):
    if titlesToken[i] == None:
        print(titlesToken[i])
        continue
    else:
        for u in range(len(titlesToken[i])):
            titlesToken[i][u] = lemmatizer.lemmatize(titlesToken[i][u])
    pbar.tic()
bp(titlesToken, logger)

Lemmatization   0% [                    ]
Lemmatization   9% [=                   ] (3.96s left)


None


Lemmatization  19% [===                 ] (3.48s left)


None
None


Lemmatization  29% [=====               ] (3.057s left)
Lemmatization  39% [=======             ] (2.61s left)


...

Lemmatization  69% [=============       ] (1.299s left)


None


Lemmatization  79% [===============     ] (0.865s left)


None
None


Lemmatization  89% [=================   ] (0.432s left)
[ [ u.s., energy, ..., -, bloomberg ], [ take, religion, ..., -, vox ], ..., [ white, house, ..., book, ' ], [ tapping, technology, ..., york, time ] ]


In [122]:
# build a dict with url as key, and tokenized title as value
urlTitles= dict()
for i in range(len(titlesToken)):
    urlTitles[newsList[i]] = titlesToken[i]
bp(urlTitles, 2, logger)

{
  http://13wham.com/news/local/tenants-living-without-heat-in-two-rochester-apartment-buildings: [ tenant, living, ..., |, wham ],
  http://247wallst.com/autos/2017/12/21/tax-cut-bill-expected-to-push-2018-auto-sales-higher/: [ tax, cut, ..., wall, st. ],
  http://247wallst.com/healthcare-business/2017/12/30/30-big-biotech-events-coming-in-2018/3/: [ 30, big, ..., wall, st. ],
  http://247wallst.com/special-report/2016/12/15/the-most-and-least-healthy-states-a-survey-of-all-50-2/: [ the, most, ..., wall, st. ],
  http://247wallst.com/special-report/2017/12/08/americas-25-dying-industries-2: [ america, 's, ..., wall, st. ],
  ...,
  https://zdubbzattmom.wordpress.com/2018/01/03/ex-libris-the-eyes-of-madness-presents-2017s-bakers-dozen-zakks-favorite-reads-of-the-year/r: [ rating, and, ..., of, madness ],
  https://zerohedge.com/news/2017-12-28/uber-no-longer-worlds-most-valuable-startup: [ uber, is, ..., zero, hedge ],
  https://zooksoftware.wordpress.com/2017/12/21/how-to-batch-conve

# load the vocab

In [123]:
# to filter out the words not in the vacab to shrink down the size of the query
titleVocab = []
with open('/home/yuting/PycharmProjects/data/title_vocab.txt','r') as file:
    for line in file.readlines():
        titleVocab.append(line.strip('\n'))

bp(titleVocab,2)

[ !, #, ..., •, … ]


In [124]:
len(trainUsers.keys())

500

# build user history

In [125]:
# for multiple users
trainUserQuery = dict()
pbar = ProgressBar(len(trainUsers.keys()), logger=logger, message="build user history")

for usr in trainUsers.keys():
    queryFinal = []
    
    # choose 15 urls
    if len(list(trainUsers[usr])) < 15:
        query = list(trainUsers[usr])
    else:
        query = random.sample(list(trainUsers[usr]),15)
        
    # put 15 titles together, using extend
    temp = []
    for url in query:
        if urlTitles[url] == None:
            continue
        else:
            temp.extend(urlTitles[url])
            
    # filter what is not in the vocab
    tempCompact = []
    for i in range(len(temp)):
        if temp[i] in titleVocab:
            tempCompact.append(temp[i])
    
    # remove duplicated & detokenize
    queryCompact = detokenize(list(set(tempCompact)))
    
    queryFinal.append(queryCompact)
    trainUserQuery[usr] = queryFinal
    
    pbar.tic()

build user history   0% [                    ]
build user history  10% [==                  ] (19.709s left)
build user history  20% [====                ] (17.479s left)
build user history  30% [======              ] (15.306s left)
build user history  40% [========            ] (13.125s left)
build user history  50% [==========          ] (10.94s left)
build user history  60% [============        ] (8.74s left)
build user history  70% [==============      ] (6.557s left)
build user history  80% [================    ] (4.372s left)
build user history  90% [==================  ] (2.185s left)
build user history 100% [====================] (total duration: 21.86s, mean duration: 0.043s)


# rankings

In [126]:
def computeSimilarity(p,h,y):
    
    tf.reset_default_graph() 
    model = Graph()
    saver = tf.train.Saver()

    with tf.Session()as sess:
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, '/home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt')
        loss, acc, prediction = sess.run([model.loss, model.acc, model.prediction],
                             feed_dict={model.p: p,
                                        model.h: h,
                                        model.y: y,
                                        model.keep_prob: 1})

        #print('loss: ', loss, ' acc:', acc)
        return prediction
#logit = computeSimilarity(p,h,y)

In [127]:
# # rankings of the candidates
# pbar = ProgressBar(len(trainUsers.keys()), logger=logger, message="testing")
# rankings = dict()
# for usr in trainUsers.keys():
#     usrRankings = []
    
#     # get historical data as query(p)
#     query = trainUserQuery[usr] * 1000
    
#     for candidates in evalData['candidates'][usr]:
#         candidates = list(candidates)
#         # get 1000 h as a list(h)
#         h_temp = []
#         for url in candidates:
#             h_temp.append(detokenize(urlTitles[url]))
#         # get p,h
#         p,h = char_index(query,h_temp)
#         y = np.zeros((1000))                    # whatever initialization
        
#         scoresCandidates = []
#         logit = computeSimilarity(p,h,y)            # get the 1000 scores list
#         scoresCandidates = zip(candidates,list(logit[:,1]))
#         ranking = sortBy(scoresCandidates, index=1, desc=True)
#         ranking = [e[0] for e in ranking]
#         usrRankings.append(ranking)
#     rankings[usr] = usrRankings
    
#     pbar.tic()

In [128]:
# rankings of the candidates
pbar = ProgressBar(len(trainUsers.keys()), logger=logger, message="testing")
rankings = dict()
for usr in trainUsers.keys():
    usrRankings = []
    
    # get historical data as query(p)
    query = trainUserQuery[usr] * 1000
    
    for candidates in evalData['candidates'][usr]:
        candidates = list(candidates)
        # get 1000 h as a list(h)
        h_temp = []
        for url in candidates:
            h_temp.append(detokenize(urlTitles[url]))
        # get p,h
        p,h = char_index(query,h_temp)
        y = np.zeros((1000))                    # whatever initialization
        
        scoresCandidates = []
        prediction = computeSimilarity(p,h,y)            # get the 1000 scores list
        scoresCandidates = zip(candidates,prediction)
        ranking = sortBy(scoresCandidates, index=1, desc=True)
        ranking = [e[0] for e in ranking]
        usrRankings.append(ranking)
    rankings[usr] = usrRankings
    
    pbar.tic()

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt


testing   0% [                    ]


INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfn

testing  10% [==                  ] (4h 55m 33.689s left)


INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfn

...

testing  80% [================    ] (1h 6m 10.072s left)


INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfn

testing  90% [==================  ] (33m 6.236s left)


INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_49.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfn

testing 100% [====================] (total duration: 5h 31m 13.43s, mean duration: 39.746s)


In [129]:
bp(rankings)

{
  100024324: 
  [
    [
      https://www.reuters.com/article/uk-southkorea-bitcoin/south-korea-plans-to-ban-cryptocurrency-tradin,
      http://calgaryherald.com/pmn/entertainment-pmn/indigenous-filmmaking-set-to-rise-in-canada-in-2018-a,
      ...,
      http://www.wrcbtv.com/story/37054326/county-schools-ask-for-your-input-on-education,
      https://www.pri.org/stories/2018-01-03/north-korea-reopens-border-hotline-south?utm_source=Twitter&u
    ]
  ],
  100585750: 
  [
    [
      https://medium.com/@jean/2018-new-years-liberations-4c8e9b390dcd,
      http://wapo.st/2l943KU,
      ...,
      https://tinyurl.com/y7wo47l8,
      http://thehill.com/opinion/healthcare/368588-work-requirements-will-withdraw-medicaids-hand-up-assis
    ]
  ],
  1007118306: 
  [
    [
      http://milforddailynews.com/news/20180111/franklin-meeting-to-share-with-residents-how-local-governm,
      http://www.bbc.co.uk/news/world-asia-42630473,
      ...,
      https://www.msn.com/en-us/music/news/charlie

# add the ranking

In [130]:
# check the ranking before adding to the db for evaluation
checkRankings(rankings,evalData['candidates'],maxUsers=None)

In [131]:
addRanking(modelName, rankings, config, logger=logger)

# news classification

In [132]:
checkRankings(rankings,evalData['candidates'],maxUsers=None)

In [133]:
modelName

'Convnet'